In [21]:
import  pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

In [22]:
sales_points = pd.read_csv("D:\\Competitions\\DataStorm-2022\\data\\SalesPointsV1.csv")

In [23]:
sales_points

,Unnamed: 0,ItemCode,#sales,stage,DailySales
0,0,3418,136,validation,"[18, 3, 8, 2, 4, 2, 8, 12, 0, 14, 2, 9, 8, 5, ..."
1,1,3427,136,validation,"[2, 7, 2, 3, 9, 0, 5, 8, 10, 5, 5, 4, 0, 4, 3,..."
2,2,7666,135,validation,"[14, 58, 12, 8, 5, 33, 11, 23, 24, 15, 41, 20,..."
3,3,9925,133,test,"[1, 3, 1, 2, 0, 0, 2, 1, 0, 1, 8, 0, 2, 0, 0, ..."
4,4,16936,136,test,"[4, 1, 4, 1, 0, 0, 21, 2, 1, 0, 5, 9, 2, 8, 8,..."
...,...,...,...,...,...
189,189,1101769,136,test,"[4, 1, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
190,190,1103056,122,test,"[3, 3, 2, 0, 0, 2, 1, 3, 5, 1, 1, 2, 4, 1, 10,..."
191,191,1105009,114,validation,"[3, 2, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 5, 3, ..."
192,192,1105018,109,test,"[1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 4, 0, 3, 2, 2, ..."


In [24]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


def build_arima_model(lis_data,result,order=(2,4,1),show_fig=False,show_summary=False):
    model = SARIMAX(lis_data, order=order)
    model_fit = model.fit()
    if show_summary:
        # summary of fit model
        print(model_fit.summary())

    if show_fig:
        # line plot of residuals
        residuals = pd.DataFrame(model_fit.resid)
        residuals.plot()
        plt.show()
        # density plot of residuals
        residuals.plot(kind='kde')
        plt.show()
        # summary stats of residuals
        print(residuals.describe())

    result["arima_model"] = model_fit
    return result

In [25]:
selected_data_points = sales_points

In [26]:
models_list = {}
for i in tqdm(selected_data_points.index):
    result = dict()
    product_series = sales_points.iloc[i]
    result["input_data"] = product_series
    __data = eval(product_series["DailySales"])[-50:]
    build_arima_model(__data,result,show_fig=False,show_summary=False)
    model = result["arima_model"]
    fc = model.forecast(28,alpha=0.05)
    fc_series = pd.Series(fc)
    result["day_prediction"] = fc_series
    models_list[product_series["ItemCode"]] = result
    break

  0%|          | 0/194 [00:00<?, ?it/s]d:\competitions\datastorm-2022\venv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
  0%|          | 0/194 [00:00<?, ?it/s]


In [27]:
def __day_filter_values(data):
    output = {
        "ItemCode":data["input_data"]["ItemCode"]
    }

    __pred = data["day_prediction"]

    output["w1"] = __pred[6]
    output["w2"] = __pred[13]
    output["w3"] = __pred[20]
    output["w4"] = __pred[27]

    return output

def day_series_filter_out(model_data):
    output_df = {}
    for __key in tqdm(model_data.keys()):
        output_df[__key] = __day_filter_values(model_data[__key])

    output_df = pd.DataFrame.from_dict(output_df,orient='index')
    output_df = pd.melt(output_df,id_vars=["ItemCode"],value_vars=["w1","w2","w3","w4"])
    output_df.columns = ["ItemCode","Week","WeeklySales"]
    return output_df

In [28]:
day_series_filter_out(models_list)

100%|██████████| 1/1 [00:00<?, ?it/s]


,ItemCode,Week,WeeklySales
0,3418,w1,13.654994
1,3418,w2,23.583116
2,3418,w3,43.980493
3,3418,w4,72.376098
